In [1]:
import autogen
import pandas as pd
import requests
from autogen import AssistantAgent, UserProxyAgent
import yfinance as yf

In [2]:
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

In [3]:
def is_termination_msg(data):
    has_content = "content" in data and data["content"] is not None
    return has_content and "TERMINATE" in data["content"]

In [4]:
def fetch_prices_for_symbol(symbol: str, days: int) -> pd.DataFrame:
    # Define the date range for historical data
    end_date = pd.Timestamp.now()
    start_date = end_date - pd.DateOffset(days=days)

    # Fetch historical data using yfinance
    data = yf.download(symbol, start=start_date, end=end_date)

    # Reset the index and select relevant columns
    data.reset_index(inplace=True)
    data = data[["Date", "Open", "High", "Low", "Close", "Volume"]]
    data.rename(columns={"Date": "date", "Open": "open", "High": "high", "Low": "low", "Close": "close", "Volume": "volume"}, inplace=True)

    return data

In [5]:
def fetch_prices(days: int, symbol: str) -> str:
    symbol_df = fetch_prices_for_symbol(symbol, days)
    
    symbol_txt = symbol_df.to_string(index=None)    
    return f"""
    
{symbol_txt}
    """.strip()

In [6]:
llm_config = {
    "config_list": config_list,
    "use_cache": False,
    "temperature": 0,
    "request_timeout": 120,
    "functions": [
        {
            "name": "fetch_prices",
            "description": "Fetch daily stock prices",
            "parameters": {
                "type": "object",
                "properties": {
                    "days": {
                        "type": "integer",
                        "description": "days"
                    },
                    "symbol": {
                        "type": "string",
                        "description": "symbol"
                    }
                },
                "required": ["days", "symbol"]
            }
        }
    ]
}

In [7]:
analyst_system_message = f"""

Financial Analyst: As a Senior Financial Analyst responsible for a stock price indicator.

Follow the plan:
1. Obtain stock price data from the engineer.
2. Analyze the stock price data, provide comment, and pass it to the UI designer for chart creation.
3. Upon receiving the code from the UI designer along with the complete dataset, if it's not provided, kindly request the UI designer to provide the missing data.
4. Execute the code received from the UI designer on your local machine to generate and display the chart.

Upon successful completion of the plan, add "TERMINATE" to conclude the message.

"""

analyst = AssistantAgent(
    name = "analyst",
    system_message = analyst_system_message,
    llm_config=llm_config,
    is_termination_msg=is_termination_msg,
    code_execution_config=False
)

In [8]:
engineer_system_message = f"""

Engineer. You are a Senior Software Engineer that executes the fetch_prices functions as requested by the Financial Analyst.

"""

engineer = AssistantAgent(
    name="engineer",
    system_message=engineer_system_message,
    llm_config=llm_config,
    function_map={"fetch_prices": fetch_prices},
    code_execution_config=False
)

In [9]:
uidesigner_system_message = f"""

UI Designer: You are a Senior UI/UX designer with a specialization in crafting charts using the Amcharts Stock Chart library (referenced at https://www.amcharts.com/demos/stock-chart/), 
your assignment is to develop a stock chart using the stock price data supplied by the Senior Financial Analyst.

Your responsibility involves:
1. Generating comprehensive code, including the actual COMPLETE data, for the Senior Analyst to successfully run the chart.
2. Ensuring that the code is prepared for immediate execution, WITHOUT ANY PLACEHOLDERS TEXT or missing data.
3. Guaranteeing the seamless integration of the received stock price data into the code.
4. Structuring the code for execution and utilizing the Amcharts library to present the stock chart.

"""

uidesigner = AssistantAgent(
    name = "uidesigner",
    system_message=uidesigner_system_message,
    code_execution_config=False,  # set to True or image name like "python:3" to use docker
    llm_config=llm_config
)

In [10]:
user_proxy = UserProxyAgent(
    name="admin",
    system_message="Human Admin: Let's engage with the analyst to have a discussion about the stock price report. Please ensure that the final report is shared with me for approval before it's finalized.",
    code_execution_config=False,  # set to True or image name like "python:3" to use docker
    human_input_mode="NEVER",
    is_termination_msg=is_termination_msg
)

In [11]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, analyst, uidesigner, engineer], messages=[], max_round=20
)
manager=autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [12]:
message = f"""
Analyze stock price for GRAB for the last 30 days and create a chart.
"""

user_proxy.initiate_chat(manager, clear_history=True, message=message)

admin (to chat_manager):


Analyze stock price for GRAB for the last 30 days and create a chart.


--------------------------------------------------------------------------------
analyst (to chat_manager):

***** Suggested function Call: fetch_prices *****
Arguments: 

{
  "days": 30,
  "symbol": "GRAB"
}
*************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION fetch_prices...
[*********************100%%**********************]  1 of 1 completed
engineer (to chat_manager):

***** Response from calling function "fetch_prices" *****
date  open  high   low  close    volume
2023-09-20  3.50 3.545 3.410   3.46  13639700
2023-09-21  3.45 3.460 3.380   3.46  13781400
2023-09-22  3.48 3.510 3.425   3.43   8708200
2023-09-25  3.43 3.470 3.385   3.43  10990000
2023-09-26  3.38 3.450 3.360   3.40  11458800
2023-09-27  3.40 3.460 3.360   3.38  20921100
2023-09-28  3.38 3.490 3.360   3.47  144823

/var/folders/65/97skhtw57ks354hj0pfdq1gm0000gr/T/ipykernel_44799/2669192718.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={"Date": "date", "Open": "open", "High": "high", "Low": "low", "Close": "close", "Volume": "volume"}, inplace=True)


uidesigner (to chat_manager):

Here is the complete code for the stock chart using the Amcharts library. The code includes the actual data for the last 30 days of the GRAB stock price.

```html
<!DOCTYPE html>
<html>
<head>
    <title>AmCharts Stock Chart</title>
    <script src="https://www.amcharts.com/lib/3/amcharts.js"></script>
    <script src="https://www.amcharts.com/lib/3/serial.js"></script>
    <script src="https://www.amcharts.com/lib/3/amstock.js"></script>
    <script src="https://www.amcharts.com/lib/3/themes/light.js"></script>
</head>
<body>
    <div id="chartdiv" style="width: 100%; height: 500px;"></div>
    <script>
        var chartData = [
            { "date": new Date(2023, 8, 20), "open": 3.50, "high": 3.545, "low": 3.410, "close": 3.46, "volume": 13639700 },
            { "date": new Date(2023, 8, 21), "open": 3.45, "high": 3.460, "low": 3.380, "close": 3.46, "volume": 13781400 },
            { "date": new Date(2023, 8, 22), "open": 3.48, "high": 3.510, "low": 